<a href="https://colab.research.google.com/github/namanphy/END2/blob/main/S9/Assignment_9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! git clone https://github.com/namanphy/END2.git

Cloning into 'END2'...
remote: Enumerating objects: 628, done.
remote: Counting objects: 100% (628/628), done.
remote: Compressing objects: 100% (461/461), done.
remote: Total 628 (delta 316), reused 404 (delta 157), pack-reused 0
Receiving objects: 100% (628/628), 14.45 MiB | 19.32 MiB/s, done.
Resolving deltas: 100% (316/316), done.


In [2]:
cd /content/END2/S9

/content/END2/S9


## Making Datatset

In [4]:
import pandas as pd
 
df = pd.read_csv("tweets_clean.csv")
df.head()

,Unnamed: 0,tweets,labels,tweets_clean
0,0,Obama has called the GOP budget social Darwini...,1,Obama has called the GOP budget social Darwini...
1,1,"In his teen years, Obama has been known to use...",0,"In his teen years, Obama has been known to use..."
2,2,IPA Congratulates President Barack Obama for L...,0,IPA Congratulates President Barack Obama for L...
3,3,RT @Professor_Why: #WhatsRomneyHiding - his co...,0,- his connection to supporters of Critical Ra...
4,4,RT @wardollarshome: Obama has approved more ta...,1,Obama has approved more targeted assassinatio...


In [5]:
df.shape

(1364, 4)

In [6]:
df.labels.value_counts()

0    931
1    352
2     81
Name: labels, dtype: int64

In [7]:
import random
import torch, torchtext
from torchtext import data

In [8]:
# Manual Seed
SEED = 43
torch.manual_seed(SEED)

In [9]:
Tweet = torchtext.legacy.data.Field(sequential = True, tokenize = 'spacy', batch_first =True, include_lengths=True)
Label = torchtext.legacy.data.LabelField(tokenize ='spacy', is_target=True, batch_first =True, sequential =False)

In [10]:
fields = [('tweet', Tweet), ('label', Label)]

In [11]:
example = [torchtext.legacy.data.Example.fromlist([df.tweets[i],df.labels[i]], fields) for i in range(df.shape[0])] 

In [12]:
twitterDataset = torchtext.legacy.data.Dataset(example, fields)

In [13]:
(train, valid) = twitterDataset.split(split_ratio=[85, 15], random_state = random.seed(SEED))

In [14]:
len(train), len(valid)

(1159, 205)

In [15]:
vars(train.examples[11])

{'label': 1,
 'tweet': ['@sweetbay',
  'That',
  'was',
  'Paul',
  'Ryan',
  "'s",
  'budget',
  '.',
  'How',
  'did',
  'Obama',
  "'s",
  'budget',
  'do',
  '?',
  'Getting',
  'educated',
  'on',
  'the',
  'facts',
  'is',
  'the',
  'first',
  'step',
  'in',
  'losing',
  'that',
  'liberalism',
  '!']}

In [16]:
Tweet.build_vocab(train)
Label.build_vocab(train)

In [17]:
print('Size of input vocab : ', len(Tweet.vocab))
print('Size of label vocab : ', len(Label.vocab))
print('Top 10 words appreared repeatedly :', list(Tweet.vocab.freqs.most_common(10)))
print('Labels : ', Label.vocab.stoi)

Size of input vocab :  4651
Size of label vocab :  3
Top 10 words appreared repeatedly : [('Obama', 1069), (':', 783), ('#', 780), ('.', 761), (',', 598), ('"', 550), ('the', 542), ('RT', 516), ('?', 419), ('to', 400)]
Labels :  defaultdict(None, {0: 0, 1: 1, 2: 2})


In [18]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [19]:
train_iterator, valid_iterator = torchtext.legacy.data.BucketIterator.splits((train, valid), batch_size = 32, 
                                                            sort_key = lambda x: len(x.tweet),
                                                            sort_within_batch=True, device = device)

In [20]:
next(iter(train_iterator)).label
#len(train.examples[11].tweet)

tensor([0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2,
        0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')

In [21]:
import os, pickle
with open('tokenizer.pkl', 'wb') as tokens: 
    pickle.dump(Tweet.vocab.stoi, tokens)

## Building Model

In [22]:
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

In [23]:
class Encoder(nn.Module):

  def __init__(self, vocab_size, embedding_dim, hidden_dim, n_layers):
        
        super().__init__()          
        
        # Embedding layer
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        
        # LSTM layer
        self.lstm = nn.LSTM(embedding_dim, 
                           hidden_dim, 
                           num_layers=n_layers,                            
                           batch_first=True)
        
  def forward(self, text, text_lengths):
        # text = [batch size, sent_length]
        embedded = self.embedding(text)

        # embedded = [batch size, sent_len, emb dim]
        # packed sequence
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths.cpu(), batch_first=True)
        
        packed_output, (hidden, cell) = self.lstm(packed_embedded)
        #hidden = [batch size, num layers * num directions,hid dim]
        #cell = [batch size, num layers * num directions,hid dim]
            
        return (packed_output, (hidden, cell))

In [24]:
class Decoder(nn.Module):

 def __init__(self, embedding_dim, hidden_dim, output_dim, n_layers, device):
        
        super().__init__() 

        self.device = device

        # LSTM layer
        self.lstm = nn.LSTM(embedding_dim, 
                           hidden_dim, 
                           num_layers=n_layers,                            
                           batch_first=True)
        # Dense layer
        self.fc = nn.Linear(hidden_dim, output_dim)
        
 def forward(self, packed_input, hidden, cell):

        packed_input = nn.utils.rnn.PackedSequence(torch.zeros(packed_input.data.shape[0],packed_input.data.shape[1]), packed_input.batch_sizes, packed_input.sorted_indices, packed_input.unsorted_indices)
        packed_output, (hidden, cell) = self.lstm(packed_input.to(self.device), (hidden,cell))
    
        # Hidden = [batch size, hid dim * num directions]
        dense_outputs = self.fc(hidden)   
        
        # Final activation function softmax
        output = F.softmax(dense_outputs[0], dim=1)
            
        return output

In [25]:
class classifier(nn.Module):
    
    # Define all the layers used in model
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, device):
        
        super().__init__()          
        
        # LSTM layer
        self.encoder = Encoder(vocab_size, embedding_dim, hidden_dim, n_layers)
        self.decoder = Decoder(hidden_dim, hidden_dim, output_dim, n_layers, device)
        
    def forward(self, text, text_lengths):

        packed_output, (hidden, cell) = self.encoder(text, text_lengths)
        output = self.decoder(packed_output, hidden, cell)
            
        return output

In [26]:
# Define hyperparameters
size_of_vocab = len(Tweet.vocab)
embedding_dim = 300
num_hidden_nodes = 100
num_output_nodes = 3
num_layers = 1

# Instantiate the model
model = classifier(size_of_vocab, embedding_dim, num_hidden_nodes, num_output_nodes, num_layers, device)

In [27]:
model=model.to(device)
for batch in train_iterator:
  tweet = batch.tweet[0]
  length = batch.tweet[1]
  model(tweet, length)
  break

In [28]:
print(model)
 
#No. of trianable parameters
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
    
print(f'The model has {count_parameters(model):,} trainable parameters')

classifier(
  (encoder): Encoder(
    (embedding): Embedding(4651, 300)
    (lstm): LSTM(300, 100, batch_first=True)
  )
  (decoder): Decoder(
    (lstm): LSTM(100, 100, batch_first=True)
    (fc): Linear(in_features=100, out_features=3, bias=True)
  )
)
The model has 1,637,203 trainable parameters


In [29]:
import torch.optim as optim
 
# define optimizer and loss
optimizer = optim.Adam(model.parameters(), lr=2e-4)
criterion = nn.CrossEntropyLoss()
 
# define metric
def binary_accuracy(preds, y):
    #round predictions to the closest integer
    _, predictions = torch.max(preds, 1)
    
    correct = (predictions == y).float() 
    acc = (correct.sum() / len(correct)) * 100
    return acc
    
# push to cuda if available
model = model.to(device)
criterion = criterion.to(device)

In [33]:
def evaluate_metrics(function):
  from functools import wraps
  @wraps(function)
  def inner(model, iterator, criterion, optimizer = None):
    labels, preds = [], []
    pred = function(model, iterator, criterion, optimizer)
    for i,j in enumerate(pred):
      if i % 2 == 0:
        preds.append(j)
      else:
        labels.append(j.tolist()) 
    predictions = torch.cat(preds, dim=0).argmax(dim=1).tolist()
    labels = sum(labels, [])
    positive = {'True Positive' : [predictions[i] for i in np.where(np.array(labels, dtype=object) == 1)[0]].count(1), 'False Neutral' : [predictions[i] for i in np.where(np.array(labels, dtype=object) == 1)[0]].count(2), 'False Negative' : [predictions[i] for i in np.where(np.array(labels, dtype=object) == 1)[0]].count(0)}
    neutral = {'False Positive' : [predictions[i] for i in np.where(np.array(labels, dtype=object) == 2)[0]].count(1), 'True Neutral' : [predictions[i] for i in np.where(np.array(labels, dtype=object) == 2)[0]].count(2),  'False Negative' : [predictions[i] for i in np.where(np.array(labels, dtype=object) == 2)[0]].count(0)}
    negative = {'False Positive' : [predictions[i] for i in np.where(np.array(labels, dtype=object) == 0)[0]].count(1), 'False Neutral' : [predictions[i] for i in np.where(np.array(labels, dtype=object) == 0)[0]].count(2), 'True Negative' : [predictions[i] for i in np.where(np.array(labels, dtype=object) == 0)[0]].count(0)}
    
    precision = {'positive' : positive['True Positive'] / (positive['True Positive'] + neutral['False Positive'] + negative['False Positive'] + 1e-15), 'neutral' : neutral['True Neutral'] / (positive['False Neutral'] + neutral['True Neutral'] + negative['False Neutral'] + 1e-15), 'negative' : negative['True Negative'] / (positive['False Negative'] + neutral['False Negative'] + negative['True Negative'] + 1e-15)}
    recall = {'positive' : positive['True Positive'] / (positive['True Positive'] + positive['False Neutral'] + positive['False Negative'] + 1e-15), 'neutral' : neutral['True Neutral'] / (neutral['False Positive'] + neutral['True Neutral'] + neutral['False Negative'] + 1e-15), 'negative' : negative['True Negative'] / (negative['False Positive'] + negative['False Neutral'] + negative['True Negative'] + 1e-15)} 
    f1_score = {'positive' : 2 * precision['positive'] * recall['positive'] / (precision['positive'] + recall['positive'] + 1e-15), 'neutral' : 2 * precision['neutral'] * recall['neutral'] / (precision['neutral'] + recall['neutral'] + 1e-15), 'negative' : 2 * precision['negative'] * recall['negative'] / (precision['negative'] + recall['negative'] + 1e-15)}
    return precision, recall, f1_score
  return inner

In [34]:
@evaluate_metrics
def train(model, iterator, criterion, optimizer):
    
    # initialize every epoch 
    epoch_loss = 0
    epoch_acc = 0
    
    # set the model in training phase
    model.train()  
    
    for batch in iterator:
        
        # resets the gradients after every batch
        optimizer.zero_grad()   
        
        # retrieve text and no. of words
        tweet, tweet_lengths = batch.tweet  
        
        # convert to 1D tensor
        predictions = model(tweet, tweet_lengths).squeeze()  
        
        # compute the loss
        loss = criterion(predictions, batch.label)        
        
        acc = binary_accuracy(predictions, batch.label)  
        
        # backpropage the loss and compute the gradients
        loss.backward()       
        
        # update the weights
        optimizer.step() 
 
        
        # loss and accuracy
        epoch_loss += loss.item() 
        epoch_acc += acc.item()  
 
        yield predictions.cpu().detach()
        yield batch.label.cpu().detach()    
    
    print("Loss = {:.2f}".format(epoch_loss / len(iterator))) 
    print("Accuracy = {:.2f}% \n ".format(epoch_acc / len(iterator)))

In [35]:
@evaluate_metrics
def evaluate(model, iterator, criterion, optimizer=None):
    
    # initialize every epoch
    epoch_loss = 0
    epoch_acc = 0

    # deactivating dropout layers
    model.eval()
    
    # deactivates autograd
    with torch.no_grad():
    
        for batch in iterator:
        
            # retrieve text and no. of words
            tweet, tweet_lengths = batch.tweet
            
            # convert to 1d tensor
            predictions = model(tweet, tweet_lengths).squeeze()
            
            # compute loss and accuracy
            loss = criterion(predictions, batch.label)
            acc = binary_accuracy(predictions, batch.label) 
            
            # keep track of loss and accuracy
            epoch_loss += loss.item()
            epoch_acc += acc.item()  

            yield predictions.cpu().detach()
            yield batch.label.cpu().detach()

    print("Loss : {:.2f}".format(epoch_loss / len(iterator)))
    print("Accuracy = {:.2f}% \n ".format(epoch_acc / len(iterator))) 

In [36]:
N_EPOCHS = 10
best_valid_loss = float('inf')
 
for epoch in range(N_EPOCHS):
    print("\n\033[1mEpoch : {} \033[0m".format(epoch + 1))
    print(" ")
    # train the model
    print("\033[4mTrain Dataset\033[0m\n ")
    recall, precision, f1_score = train(model, train_iterator, criterion, optimizer)
    print ("{:<10} {:<10} {:<10} {:<10}".format('-','Positive','Neutral','Negative'))
    print ("{:<10} {:<10.2f} {:<10.2f} {:<10.2f}".format('Recall', recall['positive'], recall['neutral'], recall['negative']))
    print ("{:<10} {:<10.2f} {:<10.2f} {:<10.2f}".format('Precision', precision['positive'], precision['neutral'], precision['negative']))
    print ("{:<10} {:<10.2f} {:<10.2f} {:<10.2f} \n".format('F1 Score', f1_score['positive'], f1_score['neutral'], f1_score['negative']))


    # evaluate the model
    print("\033[4mValidation Dataset\033[0m\n ")
    recall, precision, f1_score = evaluate(model, train_iterator, criterion)
    print ("{:<10} {:<10} {:<10} {:<10}".format('-','Positive','Neutral','Negative'))
    print ("{:<10} {:<10.2f} {:<10.2f} {:<10.2f}".format('Recall', recall['positive'], recall['neutral'], recall['negative']))
    print ("{:<10} {:<10.2f} {:<10.2f} {:<10.2f}".format('Precision', precision['positive'], precision['neutral'], precision['negative']))
    print ("{:<10} {:<10.2f} {:<10.2f} {:<10.2f}".format('F1 Score', f1_score['positive'], f1_score['neutral'], f1_score['negative']))
    print("-----------------------------------------")


Epoch : 1 
 
Train Dataset
 
Loss = 1.09
Accuracy = 54.02% 
 
-          Positive   Neutral    Negative  
Recall     0.28       0.00       0.71      
Precision  0.41       0.00       0.65      
F1 Score   0.33       0.00       0.68       

Validation Dataset
 
Loss : 1.07
Accuracy = 69.04% 
 
-          Positive   Neutral    Negative  
Recall     0.00       0.00       0.69      
Precision  0.00       0.00       1.00      
F1 Score   0.00       0.00       0.81      
-----------------------------------------

Epoch : 2 
 
Train Dataset
 
Loss = 1.04
Accuracy = 69.12% 
 
-          Positive   Neutral    Negative  
Recall     0.00       0.00       0.69      
Precision  0.00       0.00       1.00      
F1 Score   0.00       0.00       0.81       

Validation Dataset
 
Loss : 0.94
Accuracy = 69.12% 
 
-          Positive   Neutral    Negative  
Recall     0.00       0.00       0.69      
Precision  0.00       0.00       1.00      
F1 Score   0.00       0.00       0.81      
----------------

In [38]:
#inference 
 
import spacy
nlp = spacy.load('en')
 
def classify_tweet(tweet):
    
    categories = {0: "Negative", 1:"Positive", 2:"Neutral"}
    
    # tokenize the tweet 
    tokenized = [tok.text for tok in nlp.tokenizer(tweet)] 
    # convert to integer sequence using predefined tokenizer dictionary
    indexed = [tokenizer[t] for t in tokenized]        
    # compute no. of words        
    length = [len(indexed)]
    # convert to tensor                                    
    tensor = torch.LongTensor(indexed).to(device)   
    # reshape in form of batch, no. of words           
    tensor = tensor.unsqueeze(1).T  
    # convert to tensor                          
    length_tensor = torch.LongTensor(length)
    # Get the model prediction                  
    prediction = model(tensor, length_tensor)
 
    _, pred = torch.max(prediction, 1) 
    
    return categories[pred.item()]

In [ ]:
classify_tweet("A valid explanation for why Trump won't let women on the golf course.")